In [1]:
# Copying Code from main.py

In [1]:
import torch
import sklearn
import numpy as np
import pandas as pd
from robust_losses import RobustLoss
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from torch.utils.data import Dataset
from glob import glob

import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        assert x.shape[0] == y.shape[0] # assuming shape[0] = dataset size
        self.x = x
        self.y = y


    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [3]:
class Logistic_Reg_model(torch.nn.Module):
    def __init__(self,no_input_features):
        super(Logistic_Reg_model,self).__init__()
        self.layer1=torch.nn.Linear(no_input_features,64)
        self.layer2=torch.nn.Linear(64,1)
    def forward(self,x):
        y_predicted=self.layer1(x)
        y_predicted=torch.sigmoid(self.layer2(y_predicted))
        return y_predicted

In [4]:
def testaccuracy():
    with torch.no_grad():
        y_pred=model(x_test)
        y_pred_class=y_pred.round()
        accuracy=(y_pred_class.eq(y_test).sum())/float(y_test.shape[0])
        return (accuracy.item())

In [5]:
def saveModel():
    fname = "DRO_model_emp.pth"
    torch.save(model.state_dict(), fname)

In [6]:
path = '../data/datasets/employment_tx/'
synthfols = glob("../data/datasets/employment_tx/synthetic/*/")
paths = [path]
paths.extend(synthfols)

path = '../data/datasets/employment_ny/'
synthfols = glob("../data/datasets/employment_ny/synthetic/*/")
paths.append(path)
paths.extend(synthfols)

path = '../data/datasets/employment_ca/'
synthfols = glob("../data/datasets/employment_ca/synthetic/*/")
paths.append(path)
paths.extend(synthfols)

In [ ]:
for path in paths:
    
#     if os.path.exists(path+'preds/DRO_pred.pt') == False:

    train_df = pd.read_csv(path+'train.csv',header=None)

    train_df.columns = ['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR']


#     train_df.index = train_df['SEX']

    train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    test_df = pd.read_csv(path+'test.csv',header=None)

    test_df.columns = ['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR']


#     test_df.index = test_df['SEX']

    test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


    x_train = train_df.drop(['ESR','SEX'],axis=1)
    x_test = test_df.drop(['ESR', 'SEX'],axis=1)

    x_merged = pd.concat([x_train,x_test])

    ohe = make_column_transformer(
        (OneHotEncoder(sparse=False), x_merged.dtypes == 'object'),
        remainder='passthrough', verbose_feature_names_out=False)

    x_merged_temp  = pd.DataFrame(ohe.fit_transform(x_merged), columns=ohe.get_feature_names_out(), index=x_merged.index)

    x_train  = pd.DataFrame(ohe.transform(x_train), columns=ohe.get_feature_names_out(), index=x_train.index)
    x_test = pd.DataFrame(ohe.transform(x_test), columns=ohe.get_feature_names_out(), index=x_test.index)

    y_train = pd.Series(train_df['ESR'])
    y_test = pd.Series(test_df['ESR'])

    y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
    y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

    x_train=torch.from_numpy(x_train.to_numpy().astype(np.float32))
    x_test=torch.from_numpy(x_test.to_numpy().astype(np.float32))
    y_train=torch.from_numpy(y_train.to_numpy().astype(np.float32))
    y_test=torch.from_numpy(y_test.to_numpy().astype(np.float32))

    y_train=y_train.view(y_train.shape[0],1)
    y_test=y_test.view(y_test.shape[0],1)

    traindata = MyDataset(x_train, y_train)

    trainloader = torch.utils.data.DataLoader(traindata, batch_size=1000, shuffle=True)

    n_features = x_train.shape[1]
    model=Logistic_Reg_model(n_features)

    criterion=torch.nn.BCELoss(reduction='none')
    robust_loss = RobustLoss(geometry='chi-square', size=1.0, reg=0.5)
    optimizer=torch.optim.Adam(model.parameters())#,lr=0.001, weight_decay=0.0001)

    number_of_epochs=100
    best_accuracy = 0.0

    for epoch in range(number_of_epochs):
        running_loss = 0.0
        for i, (x_b, y_b) in enumerate(trainloader, 0):
            optimizer.zero_grad()
            y_prediction=model(x_b)
            loss=robust_loss(criterion(y_prediction.squeeze(),y_b.squeeze()))
            loss.backward()
            optimizer.step()
            running_loss += loss.item() 
            if (i)%10 == 9:
                print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 10))
                running_loss = 0.0
        accuracy = testaccuracy()
        print('accuracy:', accuracy)
        print('best:', best_accuracy)
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

    finalmodel = Logistic_Reg_model(n_features)
    finalmodel.load_state_dict(torch.load('DRO_model_emp.pth'))

    with torch.no_grad():
        y_pred=finalmodel(x_test)
        y_pred_class=y_pred.round()
        try:
            os.mkdir(path+'preds/')
        except:
            pass
        torch.save(y_pred_class,path+'preds/OG_DRO_pred.pt')


[1,    10] loss: 0.808
[1,    20] loss: 0.695
[1,    30] loss: 0.661
[1,    40] loss: 0.642
[1,    50] loss: 0.631
[1,    60] loss: 0.620
[1,    70] loss: 0.615
accuracy: 0.7603842616081238
best: 0.0
[2,    10] loss: 0.604
[2,    20] loss: 0.602
[2,    30] loss: 0.592
[2,    40] loss: 0.592
[2,    50] loss: 0.586
[2,    60] loss: 0.588
[2,    70] loss: 0.583
accuracy: 0.7864368557929993
best: 0.7603842616081238
[3,    10] loss: 0.578
[3,    20] loss: 0.572
[3,    30] loss: 0.576
[3,    40] loss: 0.574
[3,    50] loss: 0.576
[3,    60] loss: 0.570
[3,    70] loss: 0.576
accuracy: 0.7912404537200928
best: 0.7864368557929993
[4,    10] loss: 0.572
[4,    20] loss: 0.567
[4,    30] loss: 0.569
[4,    40] loss: 0.565
[4,    50] loss: 0.569
[4,    60] loss: 0.562
[4,    70] loss: 0.566
accuracy: 0.7936705350875854
best: 0.7912404537200928
[5,    10] loss: 0.562
[5,    20] loss: 0.568
[5,    30] loss: 0.560
[5,    40] loss: 0.561
[5,    50] loss: 0.564
[5,    60] loss: 0.563
[5,    70] loss: 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[61,    40] loss: 0.558
[61,    50] loss: 0.563
[61,    60] loss: 0.560
[61,    70] loss: 0.558
accuracy: 0.7989827394485474
best: 0.805594801902771
[62,    10] loss: 0.559
[62,    20] loss: 0.561
[62,    30] loss: 0.560
[62,    40] loss: 0.558
[62,    50] loss: 0.555
[62,    60] loss: 0.564
[62,    70] loss: 0.563
accuracy: 0.8034473061561584
best: 0.805594801902771
[63,    10] loss: 0.559
[63,    20] loss: 0.561
[63,    30] loss: 0.559
[63,    40] loss: 0.566
[63,    50] loss: 0.556
[63,    60] loss: 0.557
[63,    70] loss: 0.561
accuracy: 0.8019779324531555
best: 0.805594801902771
[64,    10] loss: 0.555
[64,    20] loss: 0.559
[64,    30] loss: 0.563
[64,    40] loss: 0.561
[64,    50] loss: 0.564
[64,    60] loss: 0.554
[64,    70] loss: 0.560
accuracy: 0.8009042143821716
best: 0.805594801902771
[65,    10] loss: 0.563
[65,    20] loss: 0.559
[65,    30] loss: 0.558
[65,    40] loss: 0.555
[65,    50] loss: 0.558
[65,    60] loss: 0.566
[65,    70] loss: 0.559
accuracy: 0.80418199